<div style="float:left">
    <h1 style="width:600px">Assessment</h1>
    <h3 style="width:600px">CASA0002: Urban Simulation</h3>
    <h3 style="width:600px">Author: Andres Restrepo</h3>

</div>
<div style="float:right"><img width="100" src="https://github.com/jreades/i2p/raw/master/img/casa_logo.jpg" /></div>

## Required libraries

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

## Data reading

### Underground network

In [3]:
G = nx.read_graphml('Data/Inputs/london.graph')

In [4]:
type(G)

networkx.classes.graph.Graph

### OD matrix

In [13]:
OD = pd.read_csv('Data/Inputs/OD_matrix.csv',index_col='station_origin')
OD.rename_axis('station_destination', axis=1,inplace=True)

In [10]:
type(OD)

pandas.core.frame.DataFrame

### Flow data

In [15]:
london_OD_AMpeak = pd.read_csv('Data/Inputs/london_flows.csv')

In [16]:
type(london_OD_AMpeak)

pandas.core.frame.DataFrame

## Data inspection

### Underground network

In [19]:
print(nx.info(G))

Graph with 401 nodes and 467 edges


/tmp/ipykernel_74/2606185536.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(G))


In [54]:
# #check that all distances are greater or equal to euclidean distance
# for u,v, data in G.edges(data=True):
#     assert(data['length'] >= distance.euclidean(pos[u], pos[v]))

In [21]:
#if want to know what is the maximum distance between stations
max(dict(G.edges).items(), key=lambda x: x[1]['length'])

(('Wembley Park', 'Finchley Road'),
 {'length': 7266.373927496304, 'line_name': 'Metropolitan', 'flows': 55124})

In [22]:
#if want to know to which stations Baker Street is directly connected to
Baker_Street = [(u,v) for  u,v in G.edges() if u == 'Baker Street' or v == 'Baker Street']
Baker_Street

[('Bond Street', 'Baker Street'),
 ('Baker Street', 'Great Portland Street'),
 ('Baker Street', 'Edgware Road'),
 ('Baker Street', 'Finchley Road'),
 ('Baker Street', 'Marylebone'),
 ('Baker Street', "Regent's Park"),
 ('Baker Street', "St. John's Wood")]

In [23]:
#We can also check the degree of the nodes in our network and check that they make sense
deg_london = nx.degree(G)

In [24]:
df = pd.DataFrame(index = dict(deg_london).keys())
df['degree'] = dict(deg_london).values()

In [25]:
df.sort_values('degree', ascending =False).head(20)

,degree
Stratford,9
Bank and Monument,8
Baker Street,7
King's Cross St. Pancras,7
Liverpool Street,6
Green Park,6
Waterloo,6
West Ham,6
Oxford Circus,6
Canning Town,6


### OD matrix

In [7]:
OD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 401 entries, station_origin to All
dtypes: int64(400), object(1)
memory usage: 1.2+ MB


In [14]:
OD.head()

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,32,599
Acton Central,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1224
Acton Town,0,0,0,3,17,0,35,0,0,11,...,77,3,6,9,0,0,0,0,0,3745
Aldgate,0,0,0,0,0,0,0,0,0,17,...,0,0,4,8,0,0,0,0,0,2886
Aldgate East,0,0,2,0,0,0,0,0,0,20,...,24,0,0,12,0,1,0,1,0,3172


### Flow data

In [17]:
london_OD_AMpeak.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61474 entries, 0 to 61473
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   station_origin       61474 non-null  object 
 1   station_destination  61474 non-null  object 
 2   flows                61474 non-null  int64  
 3   population           61474 non-null  int64  
 4   jobs                 61474 non-null  int64  
 5   distance             61474 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 2.8+ MB


In [18]:
london_OD_AMpeak.head()

,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097
1,Abbey Road,Beckton,1,599,442,8510.121774
2,Abbey Road,Blackwall,3,599,665,3775.448872
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
4,Abbey Road,Canning Town,37,599,15428,2228.923167


## London’s underground resilience

### Centrality measures

In [29]:
df_centrality = pd.DataFrame(index=G.nodes())

deg_london =nx.degree_centrality(G)
nx.set_node_attributes(G,dict(deg_london),'degree')
df_centrality['degree'] = pd.Series(nx.get_node_attributes(G, 'degree'))

clos_london = nx.closeness_centrality(G)
nx.set_node_attributes(G,clos_london,'closeness')
df_centrality['closeness'] = pd.Series(nx.get_node_attributes(G, 'closeness'))

bet_london = nx.betweenness_centrality(G)
nx.set_node_attributes(G,bet_london,'betweenness')
df_centrality['betweenness'] = pd.Series(nx.get_node_attributes(G, 'betweenness'))



In [30]:
df_centrality

,degree,closeness,betweenness
Liverpool Street,0.0150,0.110254,0.270807
Stratford,0.0225,0.104384,0.297846
Maryland,0.0050,0.095193,0.063973
Forest Gate,0.0050,0.087489,0.059337
Manor Park,0.0050,0.080939,0.054700
...,...,...,...
Kentish Town,0.0050,0.091345,0.048872
Mornington Crescent,0.0050,0.099502,0.000000
Goodge Street,0.0050,0.099527,0.000503
Battersea Power Station,0.0025,0.084692,0.000000


#### Degree centrality

In [36]:
df_centrality['degree'].sort_values(ascending=False)

Stratford                   0.0225
Bank and Monument           0.0200
Baker Street                0.0175
King's Cross St. Pancras    0.0175
Liverpool Street            0.0150
                             ...  
Watford Junction            0.0025
Cockfosters                 0.0025
Stratford International     0.0025
Wimbledon                   0.0025
Stanmore                    0.0025
Name: degree, Length: 401, dtype: float64

#### Closeness centrality

In [37]:
df_centrality['closeness'].sort_values(ascending=False)

Green Park                  0.114778
Bank and Monument           0.113572
King's Cross St. Pancras    0.113443
Westminster                 0.112549
Waterloo                    0.112265
                              ...   
Heathrow Terminals 2 & 3    0.045096
Heathrow Terminal 4         0.045091
Watford High Street         0.044302
Heathrow Terminal 5         0.043155
Watford Junction            0.042427
Name: closeness, Length: 401, dtype: float64

#### Betweenness centrality

In [38]:
df_centrality['betweenness'].sort_values(ascending=False)

Stratford                   0.297846
Bank and Monument           0.290489
Liverpool Street            0.270807
King's Cross St. Pancras    0.255307
Waterloo                    0.243921
                              ...   
Tower Gateway               0.000000
Battersea Park              0.000000
West Croydon                0.000000
Watford Junction            0.000000
Star Lane                   0.000000
Name: betweenness, Length: 401, dtype: float64

In [53]:
df_ranking = pd.DataFrame()
rank = list(range(1, df_centrality.shape[0]+1))
df_ranking['rank'] = rank

degree_rank = df_centrality.sort_values(by='degree',ascending=False).index.values.tolist()
clos_rank = df_centrality.sort_values(by='closeness',ascending=False).index.values.tolist()
bet_rank = df_centrality.sort_values(by='betweenness',ascending=False).index.values.tolist()

df_ranking['degree'] = degree_rank
df_ranking['closeness'] = clos_rank
df_ranking['betweenness'] = bet_rank

df_ranking.head(10)


,rank,degree,closeness,betweenness
0,1,Stratford,Green Park,Stratford
1,2,Bank and Monument,Bank and Monument,Bank and Monument
2,3,Baker Street,King's Cross St. Pancras,Liverpool Street
3,4,King's Cross St. Pancras,Westminster,King's Cross St. Pancras
4,5,Liverpool Street,Waterloo,Waterloo
5,6,Green Park,Oxford Circus,Green Park
6,7,Waterloo,Bond Street,Euston
7,8,West Ham,Farringdon,Westminster
8,9,Oxford Circus,Angel,Baker Street
9,10,Canning Town,Moorgate,Finchley Road


## Spatial interaction models

## Centrality measures